<a href="https://colab.research.google.com/github/meghana-anupoju/SmartyCaps/blob/main/SmartyCaps0.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import requests
import time
import random
from datetime import datetime

BOT_TOKEN = "......................................"
BASE_URL = f"https://api.telegram.org/bot{BOT_TOKEN}/"

last_update_id = None
user_scores = {}
user_quiz = {}
user_last_sent = {}
user_progress = {}
COMMANDS = {
    "/start": "Welcome to SmartyCaps! Type /help to see available commands.",
    "/help": "Here are the available commands:\n"
             "/start - Start the bot\n"
             "/capsule - Get daily knowledge\n"
             "/quiz - Answer a fun quiz\n"
             "/progress - Track your progress\n"
             "/mood - Mood check-in\n"
             "/reward - View rewards\n"
             "/settings - Customize settings\n"
             "/about - About SmartyCaps",
    "/mood": "How are you feeling today? 😊",
    "/settings": "Customize your preferences here.",
    "/about": "SmartyCaps is your personal AI learning companion!"
}
QUIZ_QUESTIONS = [
    ("What is the place value of 5 in the number 5,732?", "5000"),
    ("What is 12 × 3?", "36"),
    ("What is the sum of 25 and 47?", "72"),
    ("Which is the smallest prime number?", "2"),
    ("What is the process by which plants make their food?", "photosynthesis"),
    ("Which gas do we breathe in for survival?", "oxygen"),
    ("Name the organ that pumps blood in our body.", "heart"),
    ("What is the main source of energy on Earth?", "sun"),
    ("Which is the capital of India?", "new delhi"),
    ("Who is known as the Father of the Nation in India?", "mahatma gandhi"),
    ("Which is the longest river in India?", "ganga"),
    ("What is the name of our national currency?", "rupee")
]

def get_updates():
    """Fetch new messages from Telegram."""
    global last_update_id
    params = {"offset": last_update_id + 1} if last_update_id else {}
    response = requests.get(BASE_URL + "getUpdates", params=params)
    return response.json()

def send_message(chat_id, text):
    """Send a message to the user."""
    requests.get(BASE_URL + f"sendMessage?chat_id={chat_id}&text={text}")

def send_daily_question(chat_id):
    """Send a daily knowledge capsule to the user."""
    today = datetime.now().date()

    if user_last_sent.get(chat_id) == today:
        return

    question, answer = random.choice(QUIZ_QUESTIONS)
    send_message(chat_id, f"📖 Daily Knowledge Capsule:\n*{question}*\n✅ Answer: {answer}")

    user_last_sent[chat_id] = today

def handle_messages():
    """Continuously check for new messages and respond."""
    global last_update_id
    while True:
        updates = get_updates()
        for update in updates.get("result", []):
            message = update.get("message", {})
            chat_id = message.get("chat", {}).get("id")
            text = message.get("text", "").strip().lower()

            send_daily_question(chat_id)

            if text == "/quiz":
                question, answer = random.choice(QUIZ_QUESTIONS)
                user_quiz[chat_id] = answer

                if chat_id not in user_progress:
                    user_progress[chat_id] = {"attempted": 0, "correct": 0}

                user_progress[chat_id]["attempted"] += 1
                send_message(chat_id, f"Quiz Time! {question}\n(Type your answer below)")

            elif chat_id in user_quiz:
                correct_answer = user_quiz.pop(chat_id)

                if text == correct_answer:
                    user_scores[chat_id] = user_scores.get(chat_id, 0) + 10
                    user_progress[chat_id]["correct"] += 1
                    send_message(chat_id, f"✅ Correct! You earned 10 points.\nYour total: {user_scores[chat_id]} points.")
                else:
                    send_message(chat_id, f"❌ Incorrect. The correct answer is: {correct_answer}\nYour total: {user_scores.get(chat_id, 0)} points.")

            elif text == "/progress":
                points = user_scores.get(chat_id, 0)
                attempted = user_progress.get(chat_id, {}).get("attempted", 0)
                correct = user_progress.get(chat_id, {}).get("correct", 0)
                send_message(chat_id, f"📊 *Your Progress:*\n"
                                          f"📝 Quizzes Attempted: {attempted}\n"
                                          f"✅ Correct Answers: {correct}\n"
                                          f"🏆 Total Reward Points: {points}")

            elif text == "/reward":
                points = user_scores.get(chat_id, 0)
                if points >= 50:
                    reward_msg = "🎉 You've earned a Bronze Badge! Keep going!"
                elif points >= 100:
                    reward_msg = "🏅 You've earned a Silver Badge! Great job!"
                elif points >= 200:
                    reward_msg = "🥇 You've earned a Gold Badge! You're a champion!"
                else:
                    reward_msg = "⭐ Keep earning points to unlock rewards!"

                send_message(chat_id, f"🏆 *Your Rewards:*\n"
                                          f"🎯 Total Points: {points}\n"
                                          f"{reward_msg}")

            elif text in COMMANDS:
                send_message(chat_id, COMMANDS[text])

            else:
                send_message(chat_id, "Sorry, I didn't understand that. Type /help for commands.")

            last_update_id = update["update_id"]

        time.sleep(3)

handle_messages()

KeyboardInterrupt: 